In [35]:
import string
import pickle
import itertools
from collections import Counter
import pandas as pd

In [36]:
#Daten laden
pos_cm = pickle.load(open('d/new_pos_message.pkl','rb'))
neg_cm = pickle.load(open('d/new_neg_message.pkl','rb'))

In [37]:
# Remove punctuation
translator=str.maketrans('','',string.punctuation)
pos_cm_final = [x.translate(translator) for x in pos_cm]
neg_cm_final = [x.translate(translator) for x in neg_cm]

In [38]:
# Preprocess
pos_cm_counter = Counter(itertools.chain(*[x.split() for x in pos_cm_final]))
neg_cm_counter = Counter(itertools.chain(*[x.split() for x in neg_cm_final]))

pos_cm_sum = sum(pos_cm_counter.values())
neg_cm_sum = sum(neg_cm_counter.values())

pos_cm_counter_clean = dict()
neg_cm_counter_clean = dict()

for k,v in pos_cm_counter.items():
    if v > 50:
        pos_cm_counter_clean[k] = v/pos_cm_sum

for k,v in neg_cm_counter.items():
    if v > 50:
        neg_cm_counter_clean[k] = v/neg_cm_sum


In [39]:
# Final results
neg_pos_term = dict()
inters = set(pos_cm_counter_clean.keys()).intersection(set(neg_cm_counter_clean.keys()))
for t in inters:
    neg_pos_term[t]=neg_cm_counter_clean[t]/pos_cm_counter_clean[t]

In [40]:
# Creating Dataframe
df = pd.DataFrame(list(neg_pos_term.items()))
# Renaming
df.columns = [['Word', 'Value']]

In [41]:
# deleting words shorter than 2 digits
def length(elem):
    return len(list(elem))
df['Character Count'] = df['Word'].apply(length)
df = df[df['Character Count'] > 1]

## Was wollen wir vergleichen?

- Welche 5 Begriffe stehen am meisten in einem positiven, welchen in einem negativen Umfeld?
- Nationen-Rating?
- Bundesrats-Rating?
- National- und Ständeratsrating?
- Parteien-Rating?
- Weltführer und andere Politiker?

## Top 5

In [42]:
df.sort_values(by='Value', ascending=False).head()

,Word,Value,Character Count
25,Gutmenschen,75.678628,11
6435,Darlehen,21.382275,8
3005,Mutti,19.140048,5
1875,Idioten,15.999769,7
7901,Pöbel,12.721212,5


In [43]:
df.sort_values(by='Value').head()

,Word,Value,Character Count
8641,Saison,0.292817,6
5213,Anbieter,0.298617,8
1335,Reform,0.305139,6
8089,Geschwindigkeit,0.314949,15
2802,Reduktion,0.323242,9


## Nationen-Rating

Auch auf Tages-Anzeiger gibt es für viele Leser offenbar "Shithole countries". Aus dem Rahmen fallen vielleicht der Vatikan und Israel. Ganz gut weg, bzw. wenig beleidigt Liechtenstein.

In [44]:
c = pd.read_csv('d/german_countries.csv', error_bad_lines=False, header=None, delimiter='\t')
countries = list(c[1])
nl = []
for elem in countries:
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False)

,Country,Value
48,Somalia,1.966358
35,Nigeria,1.888437
2,Albanien,1.840101
12,Haiti,1.750735
9,Eritrea,1.668739
57,Vatikan,1.613422
18,Israel,1.580054
58,Venezuela,1.511352
29,Libanon,1.438251
33,Marokko,1.408077


## Bundesratsrating

Interessant ist, wie oft Sommaruga beschimpft wird. Und auch Darling Doris hat einen negativen Wert. Während dem die beiden SVP-Nationalräte leicht positiv sind.

In [45]:
bundesräte = ['Berset', 'Sommaruga', 'Maurer', 
              'Cassis', 'Leuthard', 'Parmelin',
              'Ammann']
nl = []
for elem in bundesräte:
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False)

,Country,Value
1,Sommaruga,2.480060
6,Ammann,1.384973
4,Leuthard,1.232500
0,Berset,1.093930
2,Maurer,0.991270
3,Cassis,0.897653
5,Parmelin,0.887061


## National- und Ständeratsrating?

In [46]:
pol = pd.read_csv('d/pol_50 (1).csv')

In [49]:
def lastname(elem):
    elem = elem.split(' ')[0]
    if '-' in elem:
        return elem.split('-')[1]
    else: 
        return elem
pol['Lastname'] = pol['Nationalrat'].apply(lastname)
lst = list(pol['Lastname'])

nl = []
for elem in lst:
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False).drop_duplicates()

,Country,Value
53,Glättli,3.562101
9,Gmür,3.552631
0,Wehrli,2.730407
65,Badran,2.561515
63,Wermuth,2.497888
62,Sommaruga,2.480060
48,Stamm,2.420133
38,Rickli,2.368036
10,Walther,2.335381
4,Jauslin,2.182182


## Parteien-Rating

In [50]:
p = ['SVP', 'FDP', 'SP', 'CVP', 'Grüne', 'BDP', 'GLP']
nl = []
for elem in p:
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False)

,Country,Value
4,Grüne,1.489313
2,SP,1.208407
6,GLP,1.107020
0,SVP,1.052711
3,CVP,0.920487
5,BDP,0.901291
1,FDP,0.805319


## State Leaders

In [61]:
p = ['Trump', 'Erdogan', 'May', 'Macron', 'Putin', 'Merkel', 'Juncker', 'Hillary']
nl = []
for elem in p:
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False)

,Country,Value
6,Juncker,1.597448
7,Hillary,1.390102
1,Erdogan,1.301566
5,Merkel,1.293441
0,Trump,1.222669
4,Putin,1.171699
2,May,0.886186
3,Macron,0.861301
